In [1]:
# !pip install sparqlwrapper
# !pip install wikidataintegrator
# !pip install --upgrade --force-reinstall git+https://github.com/vi3k6i5/flashtext.git
# !pip install rdflib
# !pip install requests_cache
# !conda install -y -c ranaroussi yfinance
# !pip install lxml

# wiki-data

wiki-data api  
https://query.wikidata.org/  
https://sqid.toolforge.org/#/view?id=P452  

https://www.wikidata.org/w/api.php?action=wbgetclaims&entity=Q66&format=jsonfm   
https://www.wikidata.org/w/api.php?action=wbgetentities&ids=Q66&format=jsonfm 

```sparql
# 列items有ticker property
SELECT ?item ?itemLabel ?value ?valueLabel
WHERE 
{
  # wdt:P249 (ticker symbol)
  ?item wdt:P249 ?value
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 10

#
?horse wdt:P31/wdt:P279* wd:Q726 .
 wd:Q180816	
```

# Wiki

wiki-page api  
https://en.wikipedia.org/wiki/Special:ApiSandbox#action=query&prop=revisions&titles=Pet_door&rvslots=*&rvprop=content&formatversion=2

Query wiki-page  
https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=1&pageids=21721040

用wikidata-id找對應的wikipage  
https://www.wikidata.org/w/api.php?action=wbgetentities&format=xml&props=sitelinks&ids=Q19675&sitefilter=frwiki


In [15]:
# 從具有company屬性的節點開始，抓5層mentions

# 取得上市公司entities
#  方法1：掃過整個wikipedia-dump，找出具有company屬性的entities
#  方法2（採用）：query wikidata找company屬性的entities

# 取得跟公司關聯的mentions（n階層）
# 掃wikipedia-dump，從company的wiki-page開始抓裡面的mentions
#  將新加入的mentions設為next_entities，重複抓取n次（=爬n層）

%cd /workspace/entity_knowledge_in_bert
import requests
import requests_cache
requests_cache.install_cache('cache')

j = requests.get("https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&props=sitelinks&ids=Q741490").json()
a = j['entities']['Q741490']['sitelinks']['enwiki']['title'].replace(' ', '_')
a



/workspace/entity_knowledge_in_bert


'Self-driving_car'

In [ ]:
# 確認爬出來的entities
#  先確認一下entities的內容，是否足夠滿足#tag需求（ie足夠豐富、多元）
#  假如不夠，可考慮採關鍵字方式與wiki-entity-dictionary做對照

# 用flashtext從news-corpus中抓bags
#  sentence-bag: 同時出現在1句子中的entities
#  paragraph-bag: 同時出現在1段落中的entities
#  sentence-coocuur-dict
#  paragraph-coocuur-dict
#  freq-daily-count-dict


In [13]:
a = "#aaa"
a[0]

'#'

In [20]:
# dbpedia infobox
!cd /workspace/entity_knowledge_in_bert
import bz2
# import rdflib
# from rdflib import Graph

# f = "./downloads/infobox-property-definitions_lang=en.ttl.bz2"
# f = "./downloads/mappingbased-objects_lang=en.ttl.bz2"
# f = "./downloads/wikipedia-links_lang=en.ttl.bz2"
f = "./downloads/redirects_lang=en.ttl.bz2"
# f = "./downloads/enwiki-latest-pages-articles1.xml-p1p30303.bz2"
# f = "./downloads/disambiguations_en.ttl.bz2"

# g = Graph()
# g.parse(f, format="turtle"); #what is right value for the format

with bz2.BZ2File(f, "rb") as f:
    for i, l in enumerate(f):
#         if i > 10:
#             break
        line = l.decode().strip()
#         print(line)
        if line[0] == '#':
            continue
        try:
            s, p, o, _ = line.split(" ")
#             print(s, p, o)
        except ValueError:
            print(line)
        
#         if "http://dbpedia.org/resource/Google" in s or "http://dbpedia.org/resource/Google" in o:
        if "<http://dbpedia.org/resource/boeing>" in s:
            print(s, p, o)

#         if "http://dbpedia.org/resource/Google" in line_decoded:
#             print(line_decoded)
        
# #             for line in tqdm.tqdm(file):
# #                 line_decoded = line.decode().strip()
# #                 matcher_match = matcher.match(line_decoded)
# #                 if matcher_match:
# #                     if matcher_match.group(match_val) in redirects:
# #                         a_to_b[
# #                             postproc_key(matcher_match.group(match_key))
# #                         ].append(redirects[postproc_val(matcher_match.group(match_val))])
# #                     else:
# #                         a_to_b[
# #                             postproc_key(matcher_match.group(match_key))
# #                         ].append(postproc_val(matcher_match.group(match_val)))
# #         return a_to_b
# for s, p, o in g:
#     print(s, p, o)

In [13]:
# flashtext
from flashtext import KeywordProcessor

text = "Car When Sebastian Thrun started working on self driving car at Google in 2007, few people outside of the company took him seriously"

keyword_processor = KeywordProcessor()
# keyword_processor.add_keyword('Big Apple', 'New York')
# keyword_processor.add_keyword('apple', 'apple watch')
# keyword_processor.add_keyword("Google")
# keyword_processor.add_keyword("Sebastian Thrun")
keyword_processor.add_keyword("Self driving (car)")
# keyword_processor.add_keyword("cars", "car")
# keyword_processor.add_keyword("car", "car")

# keyword_processor.add_keyword("apple ", "apple-watch")

# keywords_found = keyword_processor.extract_keywords(text, span_info=True, max_cost=1)
# keywords_found = keyword_processor.extract_keywords(text, span_info=False, max_cost=1)
keywords_found = keyword_processor.extract_keywords(text, span_info=False)
keywords_found

[]

In [2]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

In [ ]:
# qeury具有stock exchange（wdt:P414）屬性的items

query = """
SELECT ?item ?itemLabel ?value ?valueLabel
WHERE 
{
  ?item wdt:P414 ?value
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

results = get_results(endpoint_url, query)
# results["results"]["bindings"]

# 將results儲存在本機
import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False)

In [17]:
%cd /workspace/flair
import json

with open('data.json',  encoding='utf-8') as f:
    data = json.load(f)

# len(results["results"]["bindings"])
data["results"]["bindings"][0]

/workspace/flair


{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q66'},
 'value': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q13677'},
 'itemLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Boeing'},
 'valueLabel': {'xml:lang': 'en',
  'type': 'literal',
  'value': 'New York Stock Exchange'}}

In [16]:
# 取得entity statements（claims）
import requests
import 

entity = 'Q66'
endpoint = f'https://www.wikidata.org/w/api.php?action=wbgetclaims&entity={entity}&format=json'

r = requests.get(endpoint)
d = r.json()
# d['claims']

In [5]:
# 取得entity statements（claims） - 用wikidataintegrator
from wikidataintegrator import wdi_core

# wdid='Q707530'
wdid="Q741490"
my_first_wikidata_item = wdi_core.WDItemEngine(wd_item_id=wdid, search_only=True)

# to check successful installation and retrieval of the data, you can print the json representation of the item
d = my_first_wikidata_item.get_wd_json_representation()

In [16]:
# d['claims']['P169'][-1]
# d['claims']
# ['mainsnak']['datavalue']['value']['id']
# my_first_wikidata_item.mrh
# d['claims']
# del d['claims']
# d

In [21]:
# %cd /workspace/entity_knowledge_in_bert/wikiextractor-wikimentions
# !python WikiExtractor.py --json --filter_disambig_pages --processes 2 --collect_links ../downloads/enwiki-latest-pages-articles1.xml-p1p30303.bz2 -o test

In [30]:
%cd /workspace/entity_knowledge_in_bert
import io
import json
import pickle
import base64
import bz2
import json

extracted_wiki_file = "./downloads/enwiki-latest-pages-articles1.xml-p1p30303/AA/wiki_00"
with open(extracted_wiki_file) as f:
    for wiki_article in f.readlines():
        # each line is a dict with keys ['id', 'url', 'title', 'text', 'internal_links']
        wiki_article = json.loads(wiki_article)
#         print(wiki_article)
#         print(wiki_article['id'] + '\n')
#         print(wiki_article['url'] + '\n')
#         print(wiki_article['title'] + '\n')
#         print(wiki_article['text'])

        # The dictionary in 'internal_links' has begin and end char offsets as keys and 
        # then a tuple of mention (i.e. the link name) and entity (i.e. Wikipedia page 
        # name). Notice, f.ex. the line with 
        # 
        # (2317, 2328) ('Love Affair', 'Love Affair (1994 film)')
        # 
        # where text 'Love Affair' was linking to the Wikipedia page 'Love Affair (1994 film)'

        charoffsets_mentions = pickle.loads(base64.b64decode(wiki_article['internal_links'].encode('utf-8')))
        mentions = [
            (char_start, char_end, mention, wiki_page_name)
            for ((char_start, char_end), (mention, wiki_page_name)) in charoffsets_mentions.items()
        ]
        print(json.dumps(mentions))
        break

In [20]:
%cd /workspace/entity_knowledge_in_bert/app
import base64
import glob
import json
import pickle

import es

# p = './downloads/enwiki-latest-pages-articles1.xml-p1p30303'

# for dirname, dirnames, filenames in os.walk(p):
#     # print path to all subdirectories first.
# #     for subdirname in dirnames:
# #         print(os.path.join(dirname, subdirname))

#     # print path to all filenames.
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

        
# fs = glob.glob('./downloads/enwiki-latest-pages-articles1.xml-p1p30303/**/*')
# len(fs)

es.connect()

for p in glob.glob('../downloads/enwiki-latest-pages-articles1.xml-p1p30303/**/*'):
    with open(p) as f:
        for ln in f.readlines():
            j = json.loads(ln)
            charoffsets_mentions = pickle.loads(base64.b64decode(j['internal_links'].encode('utf-8')))
            mentions = [
                (char_start, char_end, mention, wiki_page_name)
                for ((char_start, char_end), (mention, wiki_page_name)) in charoffsets_mentions.items()
            ]
            p = es.WikiPage(
                title = j["title"],
                uid = j["id"],
                url = j["url"],
                text = j["text"],
                mentions=json.dumps(mentions, ensure_ascii=False))
            p.save()

print('done')

/workspace/entity_knowledge_in_bert/app
done


In [9]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from elasticsearch_dsl import Q
import json
import dateutil
from collections import defaultdict

client = Elasticsearch(["es:9200"])
# s = Search(using=client, index="wiki-page")
s = Search(using=client, index="scraper-page")
q = Q('wildcard', resolved_url="*cnbc*") & Q("term", http_status=200)
# q = Q("term", title="adobe inc.")
# q = Q("match", title="adobe inc")
s = s.filter(q)
# s = s.sort('article_published_at')
# s = s[101:200]
resp = s.execute()
# resp = s.params(preserve_order=True).scan()

print(resp.hits.total)

# for hit in s.filter(q).sort('-article_published_at').scan():
# for i, hit in enumerate(s.scan()):
# {"aaa": {"2020-01-01": 3, "2020-01-04": 1}, "bbb": {...}}

count = defaultdict(lambda: defaultdict(int))
for i, hit in enumerate(resp):
    try:
        d = dateutil.parser.parse(hit.article_published_at)
        o = d.date().toordinal()
        count['a'][o] += 1
    except Exception as e:
        print(e)
    if i > 100:
        break
# print(dates)
count    

# for hit in s.query(q).execute()[:10]:
#     print(hit)
# #     doc = doc.to_dict()
#     print(hit.meta.to_dict())
#     print(hit.title)
#     print(doc["article_title"], doc['article_text'])
#     parsed = json.loads(doc["parsed"])
#     print(parsed["keywords"])
# dict(count)

{'value': 10000, 'relation': 'gte'}


defaultdict(<function __main__.<lambda>()>,
            {'a': defaultdict(int, {737073: 9, 737070: 1})})

In [5]:
from dateutil import parser
from collections import defaultdict
a = defaultdict(lambda: defaultdict(int))
a["a"]["a"]
parser.parse("2019-01-14T00:00:00")

datetime.datetime(2019, 1, 14, 0, 0)

In [12]:
x = "aa" + "\n" + "bb"
print(x)

aa
bb


In [22]:
import datetime
import time
t = datetime.datetime.now()
# t.date().timestamp()
# s = time.mktime(t.date().timetuple())
# {d: [1,2,3]}
# int(s)
# datetime.datetime.fromtimestamp(int(s))

# datetime.datetime.strptime(t)
o = t.date().toordinal()
type(o)
# t.date().fromordinal(o)

int

In [12]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from elasticsearch_dsl import Q
import json

client = Elasticsearch(['es:9200'])
s = Search(using=client, index="scraper-page-202008090431")
q = Q('wildcard', resolved_url="cnbc") & Q("term", http_status=200)

for doc in s.execute()[0:3]:
    doc = doc.to_dict()
#     print(doc["article_title"], doc['article_text'])
    parsed = json.loads(doc["parsed"])
    print(parsed["keywords"])
    

['Government spending', 'Congress', 'Apparel Retail', 'Business', 'Retail industry', 'Columbia Sportswear Co', 'Donald Trump', 'Politics', 'White House', 'business news']
['Transportation', 'Asia News', 'Detroit Auto Show', 'Geely Automobile Holdings Ltd', 'Breaking News: Major', 'business news']
['Personal loans', 'Tax planning', 'Government taxation and revenue', 'Investment strategy', 'Personal finance', 'business news']


In [18]:
# count = {'aa': 1, 'bb': 2, 'cc': 3}
bags = [['aa', 'bb'], ['cc']]
bags = [e for bag in bags for e in bag]
bags = [set(bags)]

# cooccur = {'aa': {'aa': 3, 'bb': 2, 'cc': 3}}
# for k, v in count.items():
#     coocur[k][]
# count.keys()

cooccur = dict()
for bag in bags:
    for a in bag:
        cooccur[a] = cooccur.get(a, dict())
        for b in bag:
            cooccur[a][b] = cooccur[a].get(b, 0) + 1
cooccur

{'aa': {'aa': 1, 'bb': 1, 'cc': 1},
 'bb': {'aa': 1, 'bb': 1, 'cc': 1},
 'cc': {'aa': 1, 'bb': 1, 'cc': 1}}

In [21]:
a = {1,2,3,5}
# 1 in a
# a >= b
for i in a:
    print(i)

1
2
3
5


In [13]:
%cd /workspace/entity_knowledge_in_bert/app/app
import re
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
import spacy
from flashtext import KeywordProcessor

client = Elasticsearch(["es:9200"])
s = Search(using=client, index="scraper-page")
q = Q('wildcard', resolved_url="cnbc") & Q("term", http_status=200)
# q = Q("term", title="adobe inc.")

nlp = spacy.load('en_core_web_sm')
# text = 'My first birthday was great. My 2. was even better.'
# [sent.text for sent in nlp(text).sents]


kws = {"software": ["computer software", "software", "installed programs"], "San Jose, California": ["San Jose, California", "San Jose"], "Delaware": ["Delaware"], "Adobe Flash": ["Adobe Flash", "Flash"], "Photoshop": ["Photoshop Mix", "Photoshop"]}
processor = KeywordProcessor()
processor.add_keywords_from_dict(kws)

def get_cooccur(bag: str, cooccur = dict()):
    for a in bag:
        cooccur[a] = cooccur.get(a, dict())
        for b in bag:
            cooccur[a][b] = cooccur[a].get(b, 0) + 1
    return cooccur

def count_sent(sent):
    kws = processor.extract_keywords(sent)
    atk_bags.append(kws)
    return get_cooccur(kws)


sent_cooccur = {}
atk_coocur = {}
atk_bags = []
# freq_count = 

done_urls = set()
for hit in s.query(q).execute()[:10]:
    if hit.resolved_url in done_urls:
        continue

    sents = []
    try:
        sents.append(hit.article_title)
    except:
        pass
    try:
        sents += [sent.text for sent in nlp(hit.article_text).sents]
    except:
        pass
    
    atk_bag = set()
    for sent in sents:
        bag = processor.extract_keywords(sent, span_info=False)
        atk_bag |= set(bag)
        sent_coocur = get_cooccur(bag, sent_coocur)
    atk_coocur = get_cooccur(atk_bag, atk_coocur)
    atk_bags.append(atk_bag)
    #     print(hit.article_published_at)

    done_urls.add(hit.resolved_url)
#     print(hit.resolved_url)
#     print(hit.article_parsed)
#         print(hit.article_text.split('/n'))
#     print(re.split('/n|\n', hit.article_text))
#     sents = [sent.text for sent in nlp(hit.article_text).sents]
#     sents = [sent.text for sent in nlp(hit.article_text).sents]
#     print(hit.article_text)
#     print(sents)


/workspace/entity_knowledge_in_bert/app/app
[]
{}
{}
[]
{}
{}
[]
{}
{}
[]
{}
{}
[]
{}
{}
[]
{}
{}
[]
{}
{}
[]
{}
{}
[]
{}
{}
[]
{}
{}


# Ticker to comapny names

Get all tickers
* http://markets.cboe.com/us/equities/market_statistics/symbols_traded/?mkt=byx

In [44]:
import yfinance as yf

msft = yf.Ticker("RXT")
# name = msft.info['shortName']  # 'Microsoft Corporation'
# msft.info

In [1]:
%cd /workspace/entity_knowledge_in_bert/app
import os
from dotenv import load_dotenv
load_dotenv()

os.getenv("IEXCLOUD_TOKEN")
# DATABASE_PASSWORD = os.getenv("DATABASE_PASSWORD")


/workspace/entity_knowledge_in_bert/app


'pk_286936170c0d42818fd3dc57f76ce0a0'

In [46]:
import requests
import requests_cache
requests_cache.install_cache('cache')

stock = "rxt"

r = requests.get(f"https://cloud.iexapis.com/stable/stock/{stock}/company", params=params)
j = r.json()
j

{'symbol': 'RXT',
 'companyName': 'Rackspace Technology, Inc.',
 'exchange': 'NASDAQ',
 'industry': 'Data Processing Services',
 'website': 'http://www.rackspace.com',
 'description': 'Rackspace Technology, Inc. engages in the provision of end-to-end multi-cloud technology services. The firm designs, builds and operates its customers’ cloud environments across technology platforms. It operates through the following segments: Multicloud Services, Apps and Cross Platform, and OpenStack Public Cloud. The company was founded on July 21, 2016 and is headquartered in San Antonio, TX.',
 'CEO': 'Kevin M. Jones',
 'securityName': 'Rackspace Technology, Inc.',
 'issueType': 'cs',
 'sector': 'Technology Services',
 'primarySicCode': 7374,
 'employees': None,
 'tags': ['Technology Services', 'Data Processing Services'],
 'address': '1 Fanatical Place',
 'address2': None,
 'state': 'TX',
 'city': 'San Antonio',
 'zip': '78218',
 'country': 'US',
 'phone': '1.210.312.4000'}

In [7]:
import spacy

nlp = spacy.load('en_core_web_sm')

s = "Rackspace Technology, Inc."


# for c in ",.":
#     s = s.replace(c, "")
# s = s.strip()
# s.lower()

# all_stopwords = nlp.Defaults.stop_words

# text = "Nick likes to play football, however he is not too fond of tennis."
doc = nlp(s)
# tokens_without_sw= [word for word in text_tokens if not word in all_stopwords]

# print(tokens_without_sw)
# print(doc)
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)
# [tk.text for tk in doc if (not tk.is_stop or not tk.pos_ == "PUNCT")]


Rackspace Rackspace PROPN NNP compound Xxxxx True False
Technology Technology PROPN NNP ROOT Xxxxx True False
, , PUNCT , punct , False False
Inc. Inc. PROPN NNP pobj Xxx. False False


['Rackspace', 'Technology', ',', 'Inc.']

In [12]:
def candidates(term):
    cands = [term]

    # remove punct
    for c in ",.:;":
        term = term.replace(c, "")
    term = term.strip()
    cands.append(term)

    # remove extra words
    term = term.lower()
    for w in ("inc", "limited", "ltd"):
        term = term.replace(w, "")
    term = term.strip()
    cands.append(term)

    return cands

name = 'Rackspace Technology, Inc.'
candidates(name)

['Rackspace Technology, Inc.',
 'Rackspace Technology Inc',
 'rackspace technology']

In [35]:
import requests
import requests_cache
requests_cache.install_cache('cache')

name = 'Rackspace Technology, Inc.'
api = "https://en.wikipedia.org/w/api.php"
params={
    "action": "opensearch",
    "namespace": "0",
    "search": "Microsoft Corporation",
    "limit": 1,
    "format": "json",
    "redirects": "resolve",
#     'action': 'opensearch',
#     'search':'Microsoft Corporation',
#     'limit': 1,
#     'namespace': 0, 
#     'format': 'json',
}

# print(api)
r = requests.get(api, params=params)
j = r.json()
url = j[3][0]  # 'https://en.wikipedia.org/wiki/Microsoft'
url

'https://en.wikipedia.org/wiki/Microsoft'

In [2]:
import json

filepath = "/workspace/entity_knowledge_in_bert/app/outputs/wiki_mentions.json"
with open(filepath, encoding='utf-8') as f:
    j = json.load(f)

In [19]:
import pandas as pd
# df = pd.DataFrame({"name": j['seeds'][1]})
# df.to_csv('/workspace/entity_knowledge_in_bert/app/outputs/seeds_1.csv', index=False)
len(j['seeds'][2])

653773

In [12]:
%cd /workspace/entity_knowledge_in_bert/app
import json
from collections import defaultdict
import elasticsearch
import networkx as nx

from app import es
from app import preprocess

es.connect(["es:9200"])
G = preprocess.get_wikipage_mentions(["Adobe"], depth=3)

/workspace/entity_knowledge_in_bert/app
Depth 0: 1 nodes
Depth 1: 65 nodes
Depth 2: 3153 nodes


In [2]:
import networkx as nx
G = nx.DiGraph()
G.add_node("aaa", test=1)
for n in G.nodes:
    G.nodes[n]["test"]